In [39]:
%matplotlib inline
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from torchvision import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob

In [40]:
def loadDataSet():
    # convert data to a normalized torch.FloatTensor
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

    # choose the training and test datasets
    train_data = datasets.CIFAR10('data', train=True,
                                  download=True, transform=transform)

    test_data = datasets.CIFAR10('data', train=False,
                                 download=True, transform=transform)
    return train_data, test_data

In [41]:
def loader(train_data, testdata):
    
    # number of subprocesses to use for data loading
    num_workers = 0

    # how many samples per batch to load
    batch_size = 20

    # percentage of training set to use as validation
    valid_size = 0.2

    # obtain training indices that will be used for validation
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size*num_train))
    train_idx, valid_idx = indices[split:], indices[:split]


    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # prepare data loaders (combine dataset and sampler)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                               sampler=train_sampler, num_workers=num_workers)
    valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                               sampler=valid_sampler, num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                              num_workers=num_workers)

    return train_loader, valid_loader, test_loader

In [19]:
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

def visualize(loader):
    data_iter = iter(loader)
    images, labels = data_iter.next()
    images = images.numpy()
    
    # plot the images in the batch, along with the corresponding labels
    fig = plt.figure(figsize=(25,4))
    
    # display 20 images
    for idx in range(20):
        ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
        imshow(images[idx])
        ax.set_title(classes[labels[idx]])

In [26]:
# Define the CNN architecture

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Conv Layer 1
        self.conv1 = nn.Conv2d(3,16,3,padding=1)
        # Conv Layer 2
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        # Conv Layer 3
        self.conv3 = nn.Conv2d(32,64,3,padding=1)
        
        # Max Pooling Layer
        self.pool = nn.MaxPool2d(2,2)
        
        # Linear Layers
        self.fc1 = nn.Linear(4*4*64,500)
        self.fc2 = nn.Linear(500,10)
        
        # dropout
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        # Perform convolutions on input with max pool layer and relu activation
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        # Flatten view of the convolutional layer
        x = x.view(-1,4*4*64)
        
        # Add dropout layer
        x = self.dropout(x)
        
        # Add first linear layer
        x = F.relu(self.fc1(x))
        
        # Add another dropout layer
        x = self.dropout(x)
        
        # Add second linear layer
        x = self.fc2(x)
        
        return x

In [44]:
# Train the network

def train(model, train_loader, valid_loader, criterion, optimizer):
    n_epochs = 30
    valid_loss_min = np.Inf
    
    for e in range(1,n_epochs+1):
        
        # Keep track of train and valid loss
        train_loss = 0.0
        valid_loss = 0.0
        
        # Train Model #
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
        
        
        # Validate Model #
        model.eval()
        for data, target in valid_loader:
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
        
        
        # Calculate average loss
        train_loss = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(valid_loader.sampler)
        
        # Print the values
        print('Epich: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
             e, train_loss, valid_loss))
        
        # Save model if loss has decreased
        
        if valid_loss <= valid_loss_min:
            print('Validation loss has reduced from {:.6f} to {:.6f}. Saving model ...'.format(
            train_loss, valid_loss))
            torch.save(model.state_dict(), 'model_cifar.pt')
            valid_loss_min = valid_loss

In [43]:
def test(model, test_loader, criterion):
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    model.eval()
    for data, target in test_loader:
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)
        _, pred = torch.max(output, 1)
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy())
        for i in range(20): # over batch_size
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    
    # average test loss
    test_loss = test_loss/len(test_loader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))
    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                classes[i], 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))

In [ ]:
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

train_data, test_data = loadDataSet()

train_loader, valid_loader, test_loader = loader(train_data, test_data)

visualize(train_loader)

model = Net()
print(model)

# Specify loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

if (glob.glob('model_cifar.pt')):
    model.load_state_dict(torch.load('model_cifar.pt'))
    test(model, test_loader, criterion)
else:
    train(model, train_loader, valid_loader, criterion, optimizer)